# Prerequisites

- Python 3.10.4

> Warning: Installation from conda environment may take few minutes

Configuring conda environment
```cmd
conda create -n ca2_env
conda activate ca2_env
conda install -c anaconda openpyxl
```

Installing jupyter notebook on ca2_env environment
```cmd
conda install jupyter notebook
python -m ipykernel install --name ca2_env
```

Run jupiter 
```cmd
jupyter notebook
```

How crop and cereals contributes to the agricultural value added indices in Ireland and Europe.

In [1]:
from preamble import agriculture
import pandas as pd
import numpy as np



crop_datasets = {
    "fao-crop-residues"                     : "../data/fao/EU-Crop-Residues.csv",
    "fao-crop-production-idx"               : "../data/fao/EU-Crops-Production-Indices.csv",
    "fao-employment-indicators-rural"       : "../data/fao/EU-Employment-Indicators-Rural-All.csv",
    "fao-employment-indicators-agri-hours"  : "../data/fao/EU-Employment-Indicators-Agricultural-Working-Hours.csv",
    "fao-cereals-producer-prices"           : "../data/fao/EU-Producer-Prices-Cereals.csv",
    "fao-land-use"                          : "../data/fao/EU-Land-Use.csv",
    "fao-total-energy-use"                  : "../data/fao/EU-Total-Energy-Use.csv",
    "fao-cereals-export-import-idx"             : "../data/fao/EU-Trade-Indices-Cereals-Export-Import.csv",
    #"fao-agriculture-value-added"           : "../data/fao/EU-Value-Added-Agriculture.csv",
    "fadn-subsides"                         : "../data/fadn/fadn-subsides-year-ms-crops.xlsx",
    "fadn-summary"                          : "../data/fadn/fadn-custom-summary.xlsx",
    "fadn-rented-land"                      : "../data/fadn/fadn-rented-land.xlsx",
    "eurostats-gross-value-added"           : "../data/eurostat/gross_value_added/nama_10_a10_1_Data.csv"
}

eu_country_codes = pd.read_csv("../data/eu_country_codes.csv");


> "HR" -- Coratia has been removed as it only joins EU since 2013 and it's required minimum 10 years of data for analysis

> "LU" -- Luxembourg is exclued from the analysis as it forms very small percentage of the Duch's economy, thus most agricultural indicators and surveys from FAO are reported with low or no values

> "MT" -- Malta, no data reported on fao datasets.

In [2]:
def init_df():
    # "HR" -- Coratia
    # "LU" -- Luxembourg
    # "MT" -- Malta
    eu = ["BE","BG","CY","CZ","DK","DE","EL","ES","EE","FR","HU","IE","IT","LT","LV","NL","AT","PL","PT","RO","FI","SE","SK","SI"]
    years = np.arange(start=2000,stop=2022,step=1)
    # https://www.adamsmith.haus/python/answers/how-to-get-all-element-combinations-of-two-numpy-arrays-in-python
    data = np.array(np.meshgrid(eu, years)).T.reshape(-1, 2)
    df = pd.DataFrame(data, columns=["country","year"])
    df["year"] =df.year.astype(int)
    return df

def get_fao_dataset(dataset_name, query, value_col_name, aggfunc="mean", usecols=["Area Code (ISO2)","Year","Element Code","Item","Value"]):
    df = pd.read_csv(crop_datasets[dataset_name], usecols=usecols);    
    agg_df = df.query(query).groupby(["Area Code (ISO2)","Year"])["Value"].agg(aggfunc).reset_index()
    agg_df.columns = ["country","year",value_col_name]
    return agg_df

def get_fand_dataset(dataset_name, target_column, column_name):
    # Read data
    df = pd.read_excel(crop_datasets[dataset_name]);
    df = df.query("`8 Types of Farming` == '(1) Fieldcrops'")
    df = df[["ISO2", "Year", target_column]]
    df = df.rename(columns={"ISO2":"country", "Year":"year", target_column : column_name})
    df = df.replace('-',np.NaN)
    
    # Re order columns
    columns = ['country', 'year', column_name]
    df.columns = columns
    df = df[columns]

    # Convert country code to iso-alph2
    df["country"] = df.country.apply(lambda x: eu_country_codes.query(f"`EU Code` == '{x}'")["ISO2"].values[0])
    return df

def get_eurostat_gross_value_added(indicator="Value added, gross", column = "gross_value_added"):
    # Read data
    # indicator: Value added, gross | Compensation of employees | Wages and salaries | Employers' social contributions
    df = pd.read_csv(crop_datasets["eurostats-gross-value-added"]);
    nace_r2 = "Agriculture, forestry and fishing"
    df = df.query(f"NACE_R2 == '{nace_r2}' and NA_ITEM == '{indicator}'")[["ISO2", "TIME", "Value"]]
    df.columns = ["country","year",column]
    return df
    

agriculture_df = init_df()

## FAO Datasets

In [3]:
# crop_mean_residues_kg as of the average of the different crop categories in Kg of nutrients.
agriculture_df = agriculture_df.merge(get_fao_dataset("fao-crop-residues", "`Element Code` == 72392", "crop_mean_residues_kg"), how="left")

# crop_production_idx
agriculture_df = agriculture_df.merge(get_fao_dataset("fao-crop-production-idx", "`Element Code` == 432", "crop_production_idx"), how="left")

# cereals_produce_price_usd_tonne
agriculture_df = agriculture_df.merge(get_fao_dataset("fao-cereals-producer-prices", "`Element Code` == 5532", "cereals_produce_price_usd_tonne"), how="left")

# employment_ratio_rural_areas_pct
agriculture_df = agriculture_df.merge(get_fao_dataset(dataset_name="fao-employment-indicators-rural",
                                                        value_col_name="employment_ratio_rural_areas_pct",
                                                        usecols=["Area Code (ISO2)","Year","Indicator Code","Sex", "Value"],
                                                        query="`Indicator Code` == 21069 & Sex == 'Total'"), how="left")
# female_employment_ratio_rural_areas_pct
agriculture_df = agriculture_df.merge(get_fao_dataset(dataset_name="fao-employment-indicators-rural",
                                                        value_col_name="female_employment_ratio_rural_areas_pct",
                                                        usecols=["Area Code (ISO2)","Year","Indicator Code","Sex", "Value"],
                                                        query="`Indicator Code` == 21069 & Sex == 'Female'"), how="left")
# male_employment_ratio_rural_areas_pct
agriculture_df = agriculture_df.merge(get_fao_dataset(dataset_name="fao-employment-indicators-rural",
                                                        value_col_name="male_employment_ratio_rural_areas_pct",
                                                        usecols=["Area Code (ISO2)","Year","Indicator Code","Sex", "Value"],
                                                        query="`Indicator Code` == 21069 & Sex == 'Male'"), how="left")
# mean_weekly_working_hours
agriculture_df = agriculture_df.merge(get_fao_dataset(dataset_name="fao-employment-indicators-agri-hours",
                                                        value_col_name="mean_weekly_working_hours",
                                                        usecols=["Area Code (ISO2)","Year","Indicator Code","Sex", "Value"],
                                                        query="`Indicator Code` == 21150 & Sex == 'Total'"), how="left")

# female_mean_weekly_working_hours
agriculture_df = agriculture_df.merge(get_fao_dataset(dataset_name="fao-employment-indicators-agri-hours",
                                                        value_col_name="female_mean_weekly_working_hours",
                                                        usecols=["Area Code (ISO2)","Year","Indicator Code","Sex", "Value"],
                                                        query="`Indicator Code` == 21150 & Sex == 'Female'"), how="left")

# male_mean_weekly_working_hours
agriculture_df = agriculture_df.merge(get_fao_dataset(dataset_name="fao-employment-indicators-agri-hours",
                                                        value_col_name="male_mean_weekly_working_hours",
                                                        usecols=["Area Code (ISO2)","Year","Indicator Code","Sex", "Value"],
                                                        query="`Indicator Code` == 21150 & Sex == 'Male'"), how="left")

# crop_land_use_1000ha
agriculture_df = agriculture_df.merge(get_fao_dataset("fao-land-use", "`Element Code` == 5110", "crop_land_use_1000ha"), how="left")

# agri_energy_use_tj
agriculture_df = agriculture_df.merge(get_fao_dataset("fao-total-energy-use", "`Element Code` == 72184", "agri_energy_use_tj"), how="left")

# avg_import_idx
agriculture_df = agriculture_df.merge(get_fao_dataset("fao-cereals-export-import-idx", "`Element Code` == 465", "avg_import_idx"), how="left")

# avg_export_idx
agriculture_df = agriculture_df.merge(get_fao_dataset("fao-cereals-export-import-idx", "`Element Code` == 495", "avg_export_idx"), how="left")

# value_added_agriculture (Replaced with eurostat data)
# agriculture_df = agriculture_df.merge(get_fao_dataset("fao-agriculture-value-added", "`Element Code` == 6110", "value_added_agriculture"), how="left")


### FADN Dataset

In [4]:
agriculture_df = agriculture_df.merge(get_fand_dataset("fadn-subsides", "(SE610) Total subsidies on crops (€)", "total_subsides_on_field_crops" ), how="left")
agriculture_df = agriculture_df.merge(get_fand_dataset("fadn-rented-land", "(SE030) Rented UAA (ha)",           "rented_land_ha"                ), how="left")
agriculture_df = agriculture_df.merge(get_fand_dataset("fadn-rented-land", "(SE375) Rent paid (€)",             "rent_paid"                     ), how="left")
agriculture_df = agriculture_df.merge(get_fand_dataset("fadn-rented-land", "(SE025) Total Utilised Agricultural Area (ha)", "total_uaa_ha"      ), how="left")
if "pct_rented_land_of_uaa" not in agriculture_df.columns:
    agriculture_df["pct_rented_land_of_uaa"] = agriculture_df.apply(lambda x: ((x["rented_land_ha"] / x["total_uaa_ha"] ) * 100) if x["total_uaa_ha"] != None else np.nan, axis=1)

## Eurostat

In [5]:
# current prices, million euro
agriculture_df = agriculture_df.merge(get_eurostat_gross_value_added(indicator="Value added, gross", column = "gross_value_added"), how="left")
# current prices, million euro
agriculture_df = agriculture_df.merge(get_eurostat_gross_value_added(indicator="Compensation of employees", column = "compensation_of_employees"), how="left")
# current prices, million euro
agriculture_df = agriculture_df.merge(get_eurostat_gross_value_added(indicator="Wages and salaries", column = "wages_and_salaries"), how="left")

In [6]:
# fadn data available only from 2003 
agriculture_df = agriculture_df.query("year > 2003 & year <= 2020")

# set index
agriculture_df = agriculture_df.set_index(["country","year"])

# drop all rows wit no values
agriculture_df = agriculture_df.drop(agriculture_df[agriculture_df.isna().all(axis=1) == True].index)

In [7]:
agriculture_df.shape

(391, 21)

In [8]:
agriculture_df.describe()

,crop_mean_residues_kg,crop_production_idx,cereals_produce_price_usd_tonne,employment_ratio_rural_areas_pct,female_employment_ratio_rural_areas_pct,male_employment_ratio_rural_areas_pct,mean_weekly_working_hours,female_mean_weekly_working_hours,male_mean_weekly_working_hours,crop_land_use_1000ha,agri_energy_use_tj,avg_import_idx,avg_export_idx,total_subsides_on_field_crops,rented_land_ha,rent_paid,total_uaa_ha,pct_rented_land_of_uaa
count,3.680000e+02,368.000000,389.000000,355.000000,355.000000,355.000000,391.000000,391.000000,391.000000,345.000000,368.000000,391.000000,391.000000,362.000000,362.000000,362.000000,362.000000,362.000000
mean,1.528872e+07,113.108529,211.381332,52.831746,46.297831,59.622901,41.849386,36.529974,43.991100,4729.210427,50434.437709,131.016454,117.697187,2107.383978,57.983177,8772.331492,87.938785,55.269613
std,1.929832e+07,89.687527,67.661477,6.140054,7.010838,5.961734,4.678621,4.450374,5.164347,5502.400430,61097.669621,135.940223,111.729807,4673.153990,70.611976,8419.864176,75.326488,20.742784
min,8.676492e+04,31.430000,97.000000,36.840000,30.710000,42.070000,27.570000,23.620000,28.250000,107.180000,1768.371300,18.000000,2.600000,0.000000,2.590000,188.000000,8.420000,18.469155
25%,4.152103e+06,88.040000,160.000000,49.660000,42.375000,55.910000,39.995000,34.130000,41.325000,1105.000000,6636.548325,85.083333,75.000000,56.500000,18.032500,1872.250000,43.460000,36.254794
50%,8.205717e+06,99.245929,200.000000,52.850000,46.850000,59.490000,41.580000,37.150000,43.390000,2254.600000,21891.604800,102.166667,97.333333,573.000000,33.840000,4583.500000,63.410000,52.499812
75%,1.949131e+07,112.135972,252.500000,56.885000,49.995000,64.735000,43.245000,39.495000,45.955000,9121.000000,104309.755500,134.666667,122.333333,1589.000000,74.697500,16035.000000,116.867500,72.907516
max,9.350640e+07,1394.426250,589.666667,66.820000,64.830000,73.320000,55.870000,47.660000,60.340000,19488.200000,199464.000000,1998.166667,1321.500000,38794.000000,359.580000,32463.000000,395.100000,94.916375


In [9]:
# Round decimal for Crop residues kg
agriculture_df["crop_mean_residues_kg"] = agriculture_df.crop_mean_residues_kg.round(2)

## Missing At Random MAR: 

The missing value depends on other variables, (year and country for each column in the DS)

- ‘A Review of Hot Deck Imputation for Survey Non-response’ (2011). Available at: https://search.ebscohost.com/login.aspx?direct=true&db=edsoai&AN=edsoai.ocn894435509&site=eds-live (Accessed: 14 May 2022).
  
- Mohammad H. Nadimi-Shahraki et al. (2021) ‘A Hybrid Imputation Method for Multi-Pattern Missing Data: A Case Study on Type II Diabetes Diagnosis’, Electronics, 10(3167), p. 3167. doi: 10.3390/electronics10243167.

- Ma, X. and Zhong, Q. (2016) ‘Missing value imputation method for disaster decision-making using K nearest neighbor’, Journal of Applied Statistics, 43(4), pp. 767–781. doi: 10.1080/02664763.2015.1077377.
>   The main reasons that we choose KNNI method to estimate missing data are: 
>   1. K nearest neighbor can deal with heterogeneous (i.e. mixed-attributes) data; 
>   2. K nearest neighbor is little affected by the missingness mechanism; and
>   3. K nearest neighbor can easily treat instances with multiple missing values (the occurrence of multiple missing values are more common in the process of decision-making)

In [10]:
agriculture_df.isna().sum()

crop_mean_residues_kg                      23
crop_production_idx                        23
cereals_produce_price_usd_tonne             2
employment_ratio_rural_areas_pct           36
female_employment_ratio_rural_areas_pct    36
male_employment_ratio_rural_areas_pct      36
mean_weekly_working_hours                   0
female_mean_weekly_working_hours            0
male_mean_weekly_working_hours              0
crop_land_use_1000ha                       46
agri_energy_use_tj                         23
avg_import_idx                              0
avg_export_idx                              0
total_subsides_on_field_crops              29
rented_land_ha                             29
rent_paid                                  29
total_uaa_ha                               29
pct_rented_land_of_uaa                     29
gross_value_added                           0
compensation_of_employees                   0
wages_and_salaries                          0
dtype: int64

## Imputation options.

From simplest to comlex:
- mean:         Replace with mean
- linear:       Replace with predcite value from linear regrestion (pre conditions required)
- knn:          Nearest-Neighbour Imputation Methodology

### Linear regression for infering missing values on year 2020

> Weiss, N. and Weiss, C., 2017. Introductory statistics. 10th ed. Pearson Education Limited 2017, p. 667

1. Linear regression:

$\hat{Y} = \hat{\beta}_{0} + \sum \limits _{j=1} ^{p} X_{j}\hat{\beta}_{j} $


#### Assumptions:

1. Linearity: It states that the dependent variable Y should be linearly related to independent variable X.


Karl Person, Linear Correlation Coefficient
For a set of n data points, the linear correlation coefficient, r, is defined by

$ r = \frac{{}\sum_{i=1}^{n} (x_i - \overline{x})(y_i - \overline{y})}
{S_xS_y} $

,
where sx and sy denote the sample standard deviations of the x-values and
y-values, respectively.

In python corrcoef() (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html)


## Nearest-Neighbour Imputation Methodology

Precoditions. We will confirm cluster tendency using hopkins algorith.

- J. G. Skellam and Brian Hopkins (2017) ‘A New Method for determining the Type of Distribution of Plant Individuals’, Annals of Botany, 18, pp. 213–227. Available at: https://search.ebscohost.com/login.aspx?direct=true&db=edsair&AN=edsair.doi...........6ef6ec0c85f3b83af04881659901aa52&site=eds-live (Accessed: 14 May 2022).

It acts as a statistical hypothesis test where the null hypothesis is that the data is generated by a Poisson point process and are thus uniformly randomly distributed

> - H0: Data is generated by a Poisson point process.
> - H1: Data is NOT generated by a Poisson point process.

A value close to 0 tends to indicate the data is highly clustered, random data will tend to result in values around 0.5, and uniformly distributed data will tend to result in values close to 1.

- If the value is between {0.07, ...,0.99}, the data is regularly spaced.

- If the value is around 0.5, it is random.

- If the value is between {0.1, ..., 0.33}, it has a high tendency to cluster.


In [11]:
data = pd.DataFrame()

In [12]:
agriculture_df.reset_index(inplace=True)

In [13]:
impute_values_df = agriculture_df.melt(id_vars=["country","year"]); 
impute_values_df["impute_estimator"] = "mean"
impute_values_df["impute_estimator_notes"] = "mean"
impute_values_df["impute_value"] = np.nan
impute_values_df.head(5)

,country,year,variable,value,impute_estimator,impute_estimator_notes,impute_value
0,BE,2004,crop_mean_residues_kg,5241938.18,mean,mean,NaN
1,BE,2005,crop_mean_residues_kg,4919307.84,mean,mean,NaN
2,BE,2006,crop_mean_residues_kg,4757421.75,mean,mean,NaN
3,BE,2007,crop_mean_residues_kg,4838733.48,mean,mean,NaN
4,BE,2008,crop_mean_residues_kg,5378935.69,mean,mean,NaN


In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
impute_values_df.query("not impute_value.isnull()")

,country,year,variable,value,impute_estimator,impute_estimator_notes,impute_value


In [16]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor

from IPython.display import display, Markdown
from scipy.stats import levene

for country in impute_values_df.country.unique():
    for column in agriculture_df.columns:
        # Validations
        if column in ["year","country"]:
            continue;
        # Ignore if there is no null values
        no_null_values = impute_values_df.query(f"country == '{country}' and variable == '{column}' and not value.isnull()").shape[0]
        all_values  = impute_values_df.query(f"country == '{country}' and variable == '{column}'").shape[0]
        if(no_null_values == all_values):
            continue;

        # get values for calculate estimators
        #print(f"Processing {country}, for variable {column}")
        qnotnull = f"country == '{country}' and variable == '{column}' and not value.isnull()"
        query = f"country == '{country}' and variable == '{column}' and value.isnull()"
        dataset = impute_values_df.query(qnotnull)[["year","value"]]
        
        # convert to float64
        dataset.value = dataset.value.astype("float64")

        # Linear correlation
        coef = dataset.corr().iloc[1]['year']
        has_linear_corr = abs(coef) > 0.7
        if has_linear_corr:
            impute_values_df.loc[impute_values_df.eval(query),"impute_estimator"] = "linear"
            impute_values_df.loc[impute_values_df.eval(query),"impute_estimator_notes"] = f"Person Correlation Coef: {coef}"
            
            # Linear Regresion
            dataset = impute_values_df.query(qnotnull)[["year","value"]]
            dataset.value = dataset.value.astype("float64")
            x = dataset.year
            y = dataset.value
            m = stats.linregress(x, y)
            
            null_dataset = impute_values_df.query(query)
            def linear_regresion(slope, value, intercept):
                pred = slope * value + intercept
                return pred
            
            response = null_dataset["year"].apply(lambda x: linear_regresion(m.slope, x, m.intercept))
            impute_values_df.loc[response.index,"impute_value"] = response
            
        else:
            # kMeans. Check if hopkins 
             # Check cluster tendency correlation
            coef = agriculture.hopkins(dataset, 10)
            has_cluster_tendency = coef < 0.4
            if has_cluster_tendency:
                #print(f"has cluster tendency")
                impute_values_df.loc[impute_values_df.eval(query),"impute_estimator"] = "kNN"
                impute_values_df.loc[impute_values_df.eval(query),"impute_estimator_notes"] = f"hopkins: {coef}"

                # kNN 
                dataset = impute_values_df.query(qnotnull)[["year","value"]]
                dataset.value = dataset.value.astype("float64")
                x = dataset.iloc[:,[0]]
                y = dataset.iloc[:,[1]]

                neigh = KNeighborsRegressor(n_neighbors=2)
                neigh.fit(x, y)
            
                null_dataset = impute_values_df.query(query)
                response = null_dataset["year"].apply(lambda x: float(neigh.predict([[x]])[0]))
                impute_values_df.loc[response.index,"impute_value"] = response

                #display(Markdown(f"<b>{country}-{column}</b> is highly clustered"))
            #Normality plot.
            # stats.probplot(dataset.crop_mean_residues_kg, plot=plt)
            # plt.title = country
            # plt.figure()

    

In [17]:
impute_values_df.query("impute_estimator == 'kNN'")

,country,year,variable,value,impute_estimator,impute_estimator_notes,impute_value
50,CY,2020,crop_mean_residues_kg,NaN,kNN,hopkins: 0.30616067245035256,1.902552e+05
203,IT,2020,crop_mean_residues_kg,NaN,kNN,hopkins: 0.306474578596599,2.075414e+07
254,NL,2020,crop_mean_residues_kg,NaN,kNN,hopkins: 0.3944170266633082,4.440738e+06
356,SE,2020,crop_mean_residues_kg,NaN,kNN,hopkins: 0.3213061855196627,9.261173e+06
509,ES,2020,crop_production_idx,NaN,kNN,hopkins: 0.2912758381370478,2.093770e+02
...,...,...,...,...,...,...,...
6884,LV,2020,pct_rented_land_of_uaa,NaN,kNN,hopkins: 0.37997176386526876,4.672322e+01
6953,RO,2004,pct_rented_land_of_uaa,NaN,kNN,hopkins: 0.3773745460388738,5.847513e+01
6954,RO,2005,pct_rented_land_of_uaa,NaN,kNN,hopkins: 0.3773745460388738,5.847513e+01
6955,RO,2006,pct_rented_land_of_uaa,NaN,kNN,hopkins: 0.3773745460388738,5.847513e+01


In [ ]:
impute_values_df.query("country == 'DE' and variable == 'crop_mean_residues_kg'") #18462.475

## Impute results

In [ ]:
x =  agriculture_df.query("country=='CZ' & year < 2020").year
y =  agriculture_df.query("country=='CZ' & year < 2020").crop_mean_residues_kg
sns.lmplot(x='year',y='crop_mean_residues_kg',data=agriculture_df.query("country=='CZ' & year < 2020"),fit_reg=True) 


In [ ]:
# Linear Regresion
m = stats.linregress(x, y)
pred = m.slope * 2020 + m.intercept
print(pred)

In [ ]:
# Interpolate
df.query("country=='DK'").crop_mean_residues_kg.interpolate(method='linear', limit_direction='forward')

In [ ]:
agriculture_df

In [ ]:
from sklearn.linear_model import LinearRegression
X = [x]
Y = [y]
linear_regressor = LinearRegression()
linear_regressor.fit(X, Y)
Y_pred = linear_regressor.predict(X)

plt.scatter(X, Y)
plt.plot(X, Y_pred, color='red')
plt.show()